### Predicting bounding box

This notebook tries to predict the values for the upper right corner en the under left corner of the given annotated bounding boxes. We will train a CNN that tries to predict this.

In [1]:
import numpy as np
import pandas as pd
import json
import pprint
import os
import scipy.misc as spm
import glob
from tqdm import *

In [2]:
train_set = pd.read_csv('../data/validation/train.csv')

The goal is to predict the corners, where the input is **only** the image. So the class label is not known, so I create a new dataframe where there are only two thinks known. Namely, the image and the corners. Out of these images then the corners need to be predicted.

In [3]:
labeled_classes = ['ALB', 'BET', 'DOL', 'LAG', 'SHARK', 'YFT']

df = pd.DataFrame()

for label in labeled_classes:
    with open('../data/boxes/{}_labels.json'.format(label.lower())) as data_file:
        data = json.load(data_file)
        for i in range(len(data)):
            image_name = os.path.basename(data[i]['filename'])
            for box in data[i]['annotations']:
                x, y, height, width = box['x'], box['y'], box['height'], box['width']
                df[len(df.T.index)] = [image_name, x, y, height, width]
df = df.T
#print df
df.columns = ['filename', 'x', 'y', 'height', 'width']
x_temp = df[['filename']]
y_temp = df[['x', 'y', 'height', 'width']]

In [12]:
X = x_temp.as_matrix()
Y = y_temp.as_matrix()
print(X.shape)
X_train = np.zeros((X.shape[0], 3, 128, 128))
y_train = np.zeros((Y.shape[0], 4))
#for j in tqdm(range(len(X))):
for j in tqdm(range(100)):
    arra = spm.imread(glob.glob('../data/train/*/{}'.format(X[j][0]))[0])
    warp = (128.0/arra.shape[0], 128.0/arra.shape[1])
    y_train[j,0] = int(Y[j,0]*warp[1])
    y_train[j,1] = int(Y[j,1]*warp[0])
    y_train[j,2] = int(Y[j,2]*warp[0])
    y_train[j,3] = int(Y[j,3]*warp[1])
    arra2 = spm.imresize(arra, (128,128)).T
    X_train[j,:,:,:] = arra2
    
print X_train.shape
print X_train[0,:,:,:]
print y_train[0,:]

# filename = '../data/train/{}/{}'.format(example.label, example.filename)
# image = spm.imread(filename)

  1%|          | 1/100 [00:00<00:15,  6.51it/s]

(4082, 1)


100%|██████████| 100/100 [00:03<00:00, 26.17it/s]

(4082, 3, 128, 128)
[[[ 227.  184.  193. ...,   99.   99.   99.]
  [ 225.  187.  192. ...,  100.  100.  100.]
  [ 224.  178.  194. ...,   99.  101.  101.]
  ..., 
  [ 181.  182.  184. ...,  177.  112.   80.]
  [ 173.  173.  175. ...,  155.  111.   88.]
  [ 182.  180.  174. ...,  117.  103.   94.]]

 [[ 226.  182.  198. ...,  116.  116.  116.]
  [ 225.  186.  197. ...,  116.  116.  116.]
  [ 225.  177.  196. ...,  114.  116.  116.]
  ..., 
  [ 193.  194.  196. ...,  200.  136.  104.]
  [ 185.  185.  187. ...,  188.  153.  125.]
  [ 196.  194.  187. ...,  163.  158.  148.]]

 [[ 225.  180.  196. ...,  127.  128.  128.]
  [ 224.  187.  195. ...,  122.  123.  124.]
  [ 224.  175.  191. ...,  118.  120.  122.]
  ..., 
  [ 209.  209.  212. ...,  217.  156.  122.]
  [ 200.  200.  202. ...,  221.  203.  166.]
  [ 203.  201.  195. ...,  221.  228.  212.]]]
[ 54.  34.  26.  38.]
